In [23]:
import json
import sys
import os
sys.path.append(os.environ["BUILD_WORKSPACE_DIRECTORY"])


In [1]:
import functools
import os.path
import random
import sys
import xml.etree.ElementTree
import numpy as np
import matplotlib.pyplot as plt
import skimage.data
import cv2
import PIL.Image


def load_occluders(pascal_voc_root_path):
    occluders = []
    structuring_element = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (8, 8))
    
    annotation_paths = list_filepaths(os.path.join(pascal_voc_root_path, 'Annotations'))
    for annotation_path in annotation_paths:
        xml_root = xml.etree.ElementTree.parse(annotation_path).getroot()
        is_segmented = (xml_root.find('segmented').text != '0')

        if not is_segmented:
            continue

        boxes = []
        for i_obj, obj in enumerate(xml_root.findall('object')):
            is_person = (obj.find('name').text == 'person')
            is_difficult = (obj.find('difficult').text != '0')
            is_truncated = (obj.find('truncated').text != '0')
            if not is_person and not is_difficult and not is_truncated:
                bndbox = obj.find('bndbox')
                box = [int(bndbox.find(s).text) for s in ['xmin', 'ymin', 'xmax', 'ymax']]
                boxes.append((i_obj, box))

        if not boxes:
            continue

        im_filename = xml_root.find('filename').text
        seg_filename = im_filename.replace('jpg', 'png')

        im_path = os.path.join(pascal_voc_root_path, 'JPEGImages', im_filename)
        seg_path = os.path.join(pascal_voc_root_path,'SegmentationObject', seg_filename)

        im = np.asarray(PIL.Image.open(im_path))
        labels = np.asarray(PIL.Image.open(seg_path))

        for i_obj, (xmin, ymin, xmax, ymax) in boxes:
            object_mask = (labels[ymin:ymax, xmin:xmax] == i_obj + 1).astype(np.uint8)*255
            object_image = im[ymin:ymax, xmin:xmax]
            if cv2.countNonZero(object_mask) < 500:
                # Ignore small objects
                continue

            # Reduce the opacity of the mask along the border for smoother blending
            eroded = cv2.erode(object_mask, structuring_element)
            object_mask[eroded < object_mask] = 192
            object_with_mask = np.concatenate([object_image, object_mask[..., np.newaxis]], axis=-1)
            
            # Downscale for efficiency
            object_with_mask = resize_by_factor(object_with_mask, 0.75)
            occluders.append(object_with_mask)

    return occluders


def occlude_with_objects(im, occluders):
    """Returns an augmented version of `im`, containing some occluders from the Pascal VOC dataset."""

    result = im.copy()
    width_height = np.asarray([im.shape[1], im.shape[0]])
    im_scale_factor = min(width_height) / 256
    count = np.random.randint(1, 8)

    for _ in range(count):
        occluder = random.choice(occluders)

        random_scale_factor = np.random.uniform(0.4, 1.0)
        scale_factor = random_scale_factor * im_scale_factor
        occluder = resize_by_factor(occluder, scale_factor)

        center = np.random.uniform([0,0], width_height)
        paste_over(im_src=occluder, im_dst=result, center=center)

    return result


def paste_over(im_src, im_dst, center):
    """Pastes `im_src` onto `im_dst` at a specified position, with alpha blending, in place.
    Locations outside the bounds of `im_dst` are handled as expected (only a part or none of
    `im_src` becomes visible).
    Args:
        im_src: The RGBA image to be pasted onto `im_dst`. Its size can be arbitrary.
        im_dst: The target image.
        alpha: A float (0.0-1.0) array of the same size as `im_src` controlling the alpha blending
            at each pixel. Large values mean more visibility for `im_src`.
        center: coordinates in `im_dst` where the center of `im_src` should be placed.
    """

    width_height_src = np.asarray([im_src.shape[1], im_src.shape[0]])
    width_height_dst = np.asarray([im_dst.shape[1], im_dst.shape[0]])

    center = np.round(center).astype(np.int32)
    raw_start_dst = center - width_height_src // 2
    raw_end_dst = raw_start_dst + width_height_src

    start_dst = np.clip(raw_start_dst, 0, width_height_dst)
    end_dst = np.clip(raw_end_dst, 0, width_height_dst)
    region_dst = im_dst[start_dst[1]:end_dst[1], start_dst[0]:end_dst[0]]

    start_src = start_dst - raw_start_dst
    end_src = width_height_src + (end_dst - raw_end_dst)
    region_src = im_src[start_src[1]:end_src[1], start_src[0]:end_src[0]]
    color_src = region_src[..., 0:3]
    alpha = region_src[..., 3:].astype(np.float32)/255

    im_dst[start_dst[1]:end_dst[1], start_dst[0]:end_dst[0]] = (
            alpha * color_src + (1 - alpha) * region_dst)


def resize_by_factor(im, factor):
    """Returns a copy of `im` resized by `factor`, using bilinear interp for up and area interp
    for downscaling.
    """
    new_size = tuple(np.round(np.array([im.shape[1], im.shape[0]]) * factor).astype(int))
    interp = cv2.INTER_LINEAR if factor > 1.0 else cv2.INTER_AREA
    return cv2.resize(im, new_size, fx=factor, fy=factor, interpolation=interp)


def list_filepaths(dirpath):
    names = os.listdir(dirpath)
    paths = [os.path.join(dirpath, name) for name in names]
    return sorted(filter(os.path.isfile, paths))


In [2]:
occluders = load_occluders('/home/reza_voxelsafety_com/VOCdevkit/VOC2012/')

In [3]:
from timm.data.random_erasing import RandomErasing
random_erase = RandomErasing(probability=1, min_area = 0.1, max_area = 0.2, mode='const', device='cpu')

In [6]:
from core.infra.cloud.gcs_cv2_utils import upload_cv2_image_to_gcs
from core.infra.cloud.gcs_utils import dump_to_gcs, get_storage_client
output_path = "gs://voxel-users/reza/ergo_syntheticdata_occlusion"
storage_client = get_storage_client()


In [20]:
import numpy as np
import torch
import cv2
from PIL import Image
import os
import matplotlib.pyplot as plt
from torchvision import transforms
transform = transforms.ToPILImage()

def get_occlusion(data, dict_sample, max_id, root_dir):
    annotations = data['annotations'].copy()
    annotations_new = []
    annotations_new.extend(annotations)
    save_subfolder = root_dir.split('/')[-1]
    for img_name, annos in dict_sample.items():
        sample_image = Image.open(os.path.join(root_dir, img_name))
        img_array = np.asarray(sample_image)[...,:3]
        sample_image = Image.fromarray(img_array.astype('uint8'), 'RGB')
        upload_cv2_image_to_gcs(
                     cv2.cvtColor(np.asarray(sample_image), cv2.COLOR_BGR2RGB),
                     f"{output_path}/{save_subfolder}/{img_name}",
                    storage_client=storage_client,
                )
        for i in range(2):
            max_id = max_id + 1
            Image1copy = sample_image.copy()
            image_name1 = f"occlude1_{i}_{img_name}"
            occlude_key1 = {'id': max_id, 'file_name': image_name1, 'width': sample_image.width, 'height': sample_image.height}
            Image2copy = sample_image.copy()
            data['images'].append(occlude_key1)
            image_name2 = f"occlude2_{i}_{img_name}"
            max_id = max_id + 1
            occlude_key2 = {'id': max_id, 'file_name': image_name2, 'width': sample_image.width, 'height': sample_image.height}
            data['images'].append(occlude_key2)
            for anno in annos:
                anno_new = anno.copy()
                anno_new['image_id'] = occlude_key1['id']
                annotations_new.append(anno)
                anno_new['image_id'] = occlude_key2['id']
                annotations_new.append(anno)
                if anno['category_id'] == 0:
                    #print(f"keypoints: {anno['keypoints']}")
                    box = anno['bbox']
                    actor_img = sample_image.crop((int(box[0]), int(box[1]), int(box[0] + box[2]), int(box[1] + box[3])))
                    try:
                        occluded_image1 = occlude_with_objects(np.asarray(actor_img)[...,:3], occluders)
                        occluded_image1_pil = Image.fromarray(occluded_image1.astype('uint8'), 'RGB')
                        Image1copy.paste(occluded_image1_pil, (int(box[0]), int(box[1]), int(box[0] + box[2]), int(box[1] + box[3])))
                    except:
                        pass
#                     occluded_image1 = occlude_with_objects(np.asarray(actor_img)[...,:3], occluders)
                    occluded_image2 = random_erase(transforms.ToTensor()(actor_img))
                    occluded_image2_pil = transform(occluded_image2)
                    Image2copy.paste(occluded_image2_pil, (int(box[0]), int(box[1]), int(box[0] + box[2]), int(box[1] + box[3])))
            upload_cv2_image_to_gcs(
                     cv2.cvtColor(np.asarray(Image1copy), cv2.COLOR_BGR2RGB),
                     f"{output_path}/{save_subfolder}/{image_name1}",
                    storage_client=storage_client,
                )
            upload_cv2_image_to_gcs(
                    cv2.cvtColor(np.asarray(Image2copy), cv2.COLOR_BGR2RGB),
                    f"{output_path}/{save_subfolder}/{image_name2}",
                    storage_client=storage_client,
                )


    data['annotations'] = annotations_new
    gcs_label_path = f"{output_path}/{save_subfolder}/annotations.json"
    dump_to_gcs(
            gcs_label_path,
            json.dumps(data),
            content_type="application/json",
    )
    

In [13]:
root_dir = '/home/reza_voxelsafety_com/experiments/ergonomic/ergonomic-Infinity/'
data_dirs = [x[0] for x in os.walk(root_dir)][1:]
print(data_dirs)

['/home/reza_voxelsafety_com/experiments/ergonomic/ergonomic-Infinity/_americold-savannah-bloomingdale-0002', '/home/reza_voxelsafety_com/experiments/ergonomic/ergonomic-Infinity/_americold-savannah-bloomingdale-0005', '/home/reza_voxelsafety_com/experiments/ergonomic/ergonomic-Infinity/_americold-savannah-pooler-0004', '/home/reza_voxelsafety_com/experiments/ergonomic/ergonomic-Infinity/_uscold-laredo-dock01', '/home/reza_voxelsafety_com/experiments/ergonomic/ergonomic-Infinity/_americold-savannah-pooler-0008', '/home/reza_voxelsafety_com/experiments/ergonomic/ergonomic-Infinity/_americold-savannah-pooler-0007', '/home/reza_voxelsafety_com/experiments/ergonomic/ergonomic-Infinity/_americold-savannah-bloomingdale-0007', '/home/reza_voxelsafety_com/experiments/ergonomic/ergonomic-Infinity/_americold-savannah-bloomingdale-0006', '/home/reza_voxelsafety_com/experiments/ergonomic/ergonomic-Infinity/_americold-savannah-bloomingdale-0004', '/home/reza_voxelsafety_com/experiments/ergonomic/er

In [22]:
for i in range(5, len(data_dirs)):
    print(data_dirs[i])
    data = json.load(open(f"{data_dirs[i]}/annotations.json"))
    dict_sample = {}
    for name in data['images']:
        sample = [anno for anno in data['annotations'] if anno['image_id'] == name['id']]
        dict_sample[name['file_name']] = sample
    ids = [imgdata['id'] for imgdata in data['images']]
    max_id = max(ids)
    get_occlusion(data, dict_sample, max_id, data_dirs[i])
    

/home/reza_voxelsafety_com/experiments/ergonomic/ergonomic-Infinity/_americold-savannah-pooler-0007
/home/reza_voxelsafety_com/experiments/ergonomic/ergonomic-Infinity/_americold-savannah-bloomingdale-0007
/home/reza_voxelsafety_com/experiments/ergonomic/ergonomic-Infinity/_americold-savannah-bloomingdale-0006
/home/reza_voxelsafety_com/experiments/ergonomic/ergonomic-Infinity/_americold-savannah-bloomingdale-0004
/home/reza_voxelsafety_com/experiments/ergonomic/ergonomic-Infinity/_americold-savannah-pooler-0005
/home/reza_voxelsafety_com/experiments/ergonomic/ergonomic-Infinity/_americold-savannah-pooler-0006
/home/reza_voxelsafety_com/experiments/ergonomic/ergonomic-Infinity/_americold-savannah-pooler-0009
